In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential         # The base keras Neural Network model
from keras.layers import LSTM
from keras.layers import Dense

import numpy as np
import random
import io

In [ ]:
#Load the file. 
path = keras.utils.get_file(
    "design.txt", origin="https://github.com/Amelia-330/Machine-Learning/blob/main/design.txt"
)

#Alternatively, replace with your own local file:
#path = "heavy_metal_bands.txt"

with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # We remove newlines chars for nicer display (may want to play with removing this)
print("Corpus length:", len(text))

Corpus length: 120450


In [ ]:
#Take a peek at the data
text[0:100] #first 100 characters

'      <!doctype html> <html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-them'

In [ ]:
# Process the input text into a set of sequences for training:

chars = sorted(list(set(text))) #an ordered list of all characters (alphanumeric + punctuation) appearing in the corpus
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40 #sequence length
step = 3 
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
#Each element of x is a sequence; each corresponding element of y is now the next character following that sequence
#these are represented as one-hot vectors

Total chars: 72
Number of sequences: 40137


In [ ]:
#Take a peek at the data: number of sequences X sequence length X num characters
np.shape(x)

(40137, 40, 72)

In [ ]:
np.shape(y) # number of sequences X number of characters

(40137, 72)

In [ ]:
#Build the model
num_neurons = 128

model = Sequential()
model.add(LSTM(num_neurons, return_sequences=False, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               102912    
_________________________________________________________________
dense_1 (Dense)              (None, 72)                9288      
Total params: 112,200
Trainable params: 112,200
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#Set up our optimizer and compile the model
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [ ]:
#Define a simple text-sampling function:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# train:

epochs = 40 #definitely won't have time for this many...
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("Generating text after epoch: %d" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

314/314 [==============================] - 3s 6ms/step - loss: 3.0029

Generating text after epoch: 0
...Diversity: 0.2
...Generating with seed: "ne" title="repository" aria-label="repos"
...Generated:  ">                                                                                                                                                                                                                                                                                                                                                                                                              

...Diversity: 0.5
...Generating with seed: "ne" title="repository" aria-label="repos"
...Generated:  " craps jint-ty hr-s" data-nink-ne="d-clipk--.js-conte-ser-s">            </ap>                                             </span>                       </span                        </stan> </span>     </sccipt>                        </dith fix="aruink="buot;, {&quot;: {&quot;: &quot;, &quo

KeyboardInterrupt: ignored